In [70]:
from typing import Any

from attr.validators import max_len
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector.base import BaseExampleSelector


In [26]:
chat = ChatOpenAI(model_name="gpt-4o-2024-08-06", streaming=True, callbacks=[StreamingStdOutCallbackHandler()])
# temperature 등 옵션도 지정 가능

In [27]:
examples = [

    {
        "question": "What do you know about France",
        "answer":
            """Here is what I know:,
            Capital: Paris,
            Language: French,
            Food: Wine and Cheese,
            Currency: Euro,"""

    },
    {
        "question": "What do you know about Italy",
        "answer":
            """I know this:,
            Capital: Rome,
            Language: Italian,
            Food: Pizza and Pasta,
            Currency: Euro"""

    },
    {
        "question": "What do you know about Greece",
        "answer":
            """
        I know this:,
        Capital: Athens,
        Language: Greek,
        Food: Souvlaki and Feta Cheese,
        Currency: Euro,"""

    }
]

In [62]:
class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples
    def add_example(self, example):
        self.examples.append(example)
    def select_examples(self, input_variables) :
        from random import choice
        return [choice(self.examples)]

In [63]:
example_prompt = PromptTemplate.from_template(
    ("Human:{question}\nAI:{answer}")
)

In [64]:
example_selector = RandomExampleSelector(
    examples=examples,
    
)

In [65]:
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}",
    input_variables=["country"]
)

In [68]:
prompt.format(country="Brzil")

'Human:What do you know about France\nAI:Here is what I know:,\n            Capital: Paris,\n            Language: French,\n            Food: Wine and Cheese,\n            Currency: Euro,\n\nHuman: What do you know about Brzil'